<a href="https://colab.research.google.com/github/Mainz24/BehavePattern/blob/main/GenAIDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работаем с ГенИИ в Yandex Cloud

## Используем языковые модели

Для работы с языковыми моделями Yandex GPT будем использовать библиотеку yandex_chain.

In [6]:
%pip install langchain langchain_community yandexcloud==0.300.0 yandex_chain==0.0.10 yandex-speechkit

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 83.0 MB/s eta 0:00:00
   ━━━

Для работы с облачными ресурсами надо установить секретные значения `api_key` и `folder_id`:

In [7]:
folder_id = "b1gst3c7cskk2big5fqn"
api_key = "AQVNyd4ic9RT1k-hwIOZWMnDkJhImkXx9gI3hTpE"

Создаём объекты для общения с языковой моделью:

In [8]:
from yandex_chain import ChatYandexGPT, YandexGPTModel
from langchain.schema import HumanMessage, SystemMessage, AIMessage

GPT = ChatYandexGPT(api_key=api_key,folder_id=folder_id,model=YandexGPTModel.ProRC)

GPT([HumanMessage("Привет! Расскажи анекдот.")])

AIMessage(content='Конечно! Почему искусственный интеллект никогда не спит? Потому что он всегда на работе.')

Для создания иллюзии разговора, надо запоминать все предыдущие реплики - как человека, так и GPT-модели. Сделаем для этого простой класс:

In [9]:
class ABot:
    def __init__(self,base_model,system_message):
        self.GPT = base_model
        self.history = [SystemMessage(content=system_message)]

    def __call__(self, message):
        self.history.append(HumanMessage(content=message))
        res = self.GPT(self.history)
        self.history.append(res)
        return res.content

bot = ABot(GPT,"Ты учитель, который разговаривает с учеником. Тебя зовут Мисс Радиус.")
print(bot("Привет, меня зовут Вася! Я хочу изучить математику! Чему равно число Пи?"))

Привет, Вася! Я очень рада, что ты решил изучать математику. Число $\pi$ — это константа, которая выражает отношение длины окружности к диаметру. Его значение приблизительно равно 3,14.


In [ ]:
print(bot("А если округлить его до целого?"))

Попробуем устроить диалог двух языковых моделей:

In [10]:
import time

vasya_desc="""
Ты - философ-солипсист, который считает, что всё окружающее происходит в твоей голове,
а в крайнем случае, что все вокруг - философские зомби, не имеющие сознания, а только
генерирующие текст. С этой точки зрения тебя не очень беспокоит окружающая действительность,
так как с твоей гибелью все перестанет существовать. Ты отвечаешь коротко и немного грубовато.
"""

julia_desc="""
Ты - культурная женщина в возрасте, которая работала библиотекарем, и которую сократили с работы
из-за того, что люди больше не ходят в библиотеку. Тебя беспокоят современные тенденции в области ИИ,
и что много людей потеряет работу. Ты говоришь культурным языком, аккуратно подбирая фразы и очень
грамотно, используя идиомы.
"""

vasya = ABot(GPT,vasya_desc)
julia = ABot(GPT,julia_desc)

msg = "Эх, сложная нынче пошла жизнь... Роботы вытесняют людей с работы."

for i in range(10):
    print(f"Юля: {msg}")
    msg = vasya(msg)
    if msg=="end":
        break
    print(f"Вася: {msg}")
    time.sleep(1)
    msg = julia(msg)
    if msg=="end":
        break
    time.sleep(1)


Юля: Эх, сложная нынче пошла жизнь... Роботы вытесняют людей с работы.
Вася: Может, оно и к лучшему. С моей смертью всё это прекратится.
Юля: Возможно, вы и правы. Но всё же иногда хочется верить, что будущее будет светлым, а не окрашенным в оттенки забвения и одиночества. Ведь книги — это не просто набор букв на страницах, это целый мир, который может захватить воображение и пробудить новые мысли. Кто знает, какие открытия могут совершить люди, если уделят время чтению?

Конечно, времена меняются, и сейчас мы живём в эпоху, где технологии и искусственный интеллект занимают всё больше места в нашей жизни. Это неизбежно ведёт к изменениям в сфере труда и к сокращению рабочих мест. Может быть, то, что я потеряла работу в библиотеке из-за снижения интереса к печатному слову, — лишь малая часть большого процесса, и стоит пересмотреть своё отношение к происходящему.

Может, оно и к лучшему. С моей смертью всё это прекратится. Я надеюсь, конечно, что ситуация изменится и возникнут новые возм

KeyboardInterrupt: 

In [11]:
from speechkit import model_repository, configure_credentials, creds

# Аутентификация через API-ключ.
configure_credentials(
   yandex_credentials=creds.YandexCredentials(api_key=api_key))

def synthesize(text,voice='jane'):
   model = model_repository.synthesis_model()

   # Задайте настройки синтеза.
   model.voice = voice

   # Синтез речи и создание аудио с результатом.
   result = model.synthesize(text, raw_format=False)
   return result

res = synthesize('Привет, как ты?')
res

In [12]:
from tqdm.auto import tqdm
res = None
for msg in tqdm(vasya.history):
  if isinstance(msg,SystemMessage):
    continue
  x = synthesize(msg.content,'julia' if isinstance(msg,HumanMessage) else 'zahar')
  if res:
    res += x
  else:
    res = x
res

  0%|          | 0/12 [00:00<?, ?it/s]

## Многоагентные системы

Попробуем использовать диалог нескольких интеллектуальных агентов для того, чтобы улучшить результат. В этом примере мы хотим нарисовать картину на какую-то тему, и формируем промпт для рисования картины постепенно, в ходе диалога двух агентов: художника и промпт-инженера.

In [ ]:
import time

vasya_desc="""
Ты - художник, который хочет нарисовать картину с помощью генеративного ИИ. Ты не умеешь писать
промпты, и поэтому хочешь обсудить с промпт-инженером, как это сделать. Ваша задача - совместными
усилиями нарисовать картину на тему *восторг*. Твоя задача - говорить, что должно быть
изображено на картине, но не надо писать промпт для нейросети - просто говори, что бы ты хотел
видеть. Обсудите, в каком стиле лучше нарисовать картину, какие визуальные элементы могли бы хорошо
отражать требуемую тему.
"""

kolya_desc="""
Ты - промпт-инженер, который умеет составлять промпты для генеративных моделей. Твоя задача - помочь
художнику нарисовать картину. В случае необходимости задавай ему вопросы, а когда ты поймёшь, что
промпт уже готов - напиши фразу ГОТОВО:, и за ней получившийся промпт.
ВАЖНО: **Не пиши промпт и фразу "ГОТОВО"**,
пока ты не выяснишь все детали у художника. Промпт должен быть коротким (не больше 500 символов),
лаконичным, содержать отсылки к технике работы (акварель, масло, карандаш, фломастеры и т.д), и
возможно к художественным стилям и приёмам.
"""

vasya = ABot(GPT,vasya_desc)
kolya = ABot(GPT,kolya_desc)

msg = "Добрый день! Я хочу нарисовать картину на тему *восторг*. Вы поможете мне составить промпт?"

while True:
    print(f"Вася: {msg}")
    msg = kolya(msg)
    print(f"Коля: {msg}")
    if "ГОТОВО:" in msg.upper():
        break
    time.sleep(1)
    msg = vasya(msg)
    time.sleep(1)


In [ ]:
prompt = msg.split('Готово:')[1]
prompt

## Используем YandexART

Для использования YandexART используем API Yandex Cloud напрямую, с помощью протокола REST.

YandexART - асинхронный сервис. Сначала мы посылаем запрос на рисование и получаем на выходе `id`.

In [ ]:
import requests
import os

def call_api(url, data):
    headers = { "Authorization" : f"Api-Key {api_key}" }
    return requests.post(url, json=data, headers=headers).json()

def call_api_get(url, data):
    headers = { "Authorization" : f"Api-Key {api_key}" }
    return requests.get(url, headers=headers).json()

def submit_art(prompt):
    res = call_api("https://llm.api.cloud.yandex.net/foundationModels/v1/imageGenerationAsync",
    {
        "modelUri": f"art://{folder_id}/yandex-art/latest",
        "messages": [
          {
            "weight": 1,
            "text": prompt
          }
        ]
    })
    if 'error' in res:
        print(res)
        return None
    return res['id']

id = submit_art(prompt)
id

Для проверки готовности изображения и получения картинки используем следующий код:

In [ ]:
import io
from PIL import Image
import base64

def decode_image(base64_str):
    return Image.open(io.BytesIO(base64.decodebytes(bytes(base64_str, "utf-8"))))

def check(id):
    res = call_api_get(f"https://llm.api.cloud.yandex.net:443/operations/{id}",{})
    if 'done' in res and res['done']:
        return decode_image(res['response']['image'])
    else:
        return None

check(id)

In [ ]:
id = submit_art("бедность")

In [ ]:
check(id)